In [ ]:
import pandas as pd

In [ ]:
ces_df = pd.read_excel("/content/cesvin00.xlsx", sheet_name="CSV data")
ces_df.head()

,year,month,Jan_39,Feb_39,Mar_39,Apr_39,May_39,Jun_39,Jul_39,Aug_39,...,Apr_24,May_24,Jun_24,Jul_24,Aug_24,Sep_24,Oct_24,Nov_24,Dec_24,Jan_25
0,2003,5,29923,30100,30280,30094,30300,30502,30419,30662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003,6,29923,30100,30280,30094,30300,30502,30419,30662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003,7,29923,30100,30280,30094,30300,30502,30419,30662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003,8,29923,30100,30280,30094,30300,30502,30419,30662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003,9,29923,30100,30280,30094,30300,30502,30419,30662,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
qcew_df = pd.read_csv("/content/qcew-revisions.csv")
qcew_df.head()

,Year,Quarter,Area,Field,Initial Value,First Revised Value,Second Revised Value,Third Revised Value,Fourth Revised Value,Final Value
0,2017,1,United States,Quarterly Establishments,9864197,9818898,9739395,9727591,9718391,9718391
1,2017,1,United States,January Employment,140803178,140784533,140757063,140760789,140743089,140743089
2,2017,1,United States,February Employment,141534347,141527436,141505486,141511022,141509780,141509780
3,2017,1,United States,March Employment,142309185,142328256,142302986,142310827,142293796,142293796
4,2017,1,United States,Total Quarterly Wages,2044748205052,2044758442298,2044336634122,2044365522525,2044527749562,2044527749562


In [ ]:
months = []
years = range(2019, 2026)   # або до потрібного року
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for y in years:
    for m in month_names:
        months.append(f"{m}_{str(y)[-2:]}")

In [ ]:
data = []
for month in months:
    try:
        idx = ces_df[month].first_valid_index()
        if idx is not None:
            initial = ces_df.loc[idx, month]
            rev1 = ces_df.loc[idx+1, month] if (idx+1 in ces_df.index) else None
            rev2 = ces_df.loc[idx+2, month] if (idx+2 in ces_df.index) else None
            data.append({'Month': month, 'Initial': initial, '1st_revision': rev1, '2nd_revision': rev2})
    except KeyError:
        continue


triangle_extract = pd.DataFrame(data)

triangle_extract[['Month', 'Year']] = triangle_extract['Month'].str.split('_', expand=True)
triangle_extract['Year'] = triangle_extract['Year'].apply(lambda x: '20' + x)


triangle_extract.head()

,Month,Initial,1st_revision,2nd_revision,Year
0,Jan,150574.0,150586.0,150587.0,2019
1,Feb,150606.0,150620.0,150643.0,2019
2,Mar,150816.0,150832.0,150796.0,2019
3,Apr,151095.0,151020.0,151012.0,2019
4,May,151095.0,151084.0,151074.0,2019


In [ ]:
mask = (qcew_df["Area"] == "United States") & (qcew_df["Field"].str.contains("Employment")) & (qcew_df["Year"] >= 2019)
qcew_us = qcew_df[mask].reset_index(drop=True)
qcew_us.head()

,Year,Quarter,Area,Field,Initial Value,First Revised Value,Second Revised Value,Third Revised Value,Fourth Revised Value,Final Value
0,2019,1,United States,January Employment,145328396,145308900,145300240,145312731,145328097,145328097
1,2019,1,United States,February Employment,145916313,145908864,145904676,145916906,145941645,145941645
2,2019,1,United States,March Employment,146497599,146513849,146514210,146527125,146553073,146553073
3,2019,2,United States,April Employment,147522541,147536295,147543057,147574087,Not applicable,147574087
4,2019,2,United States,May Employment,148607380,148633086,148646236,148673869,Not applicable,148673869


In [ ]:
month_map = {
    "January": "Jan",
    "February": "Feb",
    "March": "Mar",
    "April": "Apr",
    "May": "May",
    "June": "Jun",
    "July": "Jul",
    "August": "Aug",
    "September": "Sep",
    "October": "Oct",
    "November": "Nov",
    "December": "Dec"
}

def extract_month(field):
    for m in month_map:
        if field.startswith(m):
            return month_map[m]
    return None

qcew_us["Month"] = qcew_us["Field"].apply(extract_month)
qcew_us["Year"] = qcew_us["Year"].astype(str)


qcew_final = qcew_us[["Year", "Month", "Final Value"]].copy()
qcew_final.head()

,Year,Month,Final Value
0,2019,Jan,145328097
1,2019,Feb,145941645
2,2019,Mar,146553073
3,2019,Apr,147574087
4,2019,May,148673869


In [ ]:
final_df = pd.merge(triangle_extract, qcew_final, on=["Year", "Month"], how="left")
final_df.head()

,Month,Initial,1st_revision,2nd_revision,Year,Final Value
0,Jan,150574.0,150586.0,150587.0,2019,145328097
1,Feb,150606.0,150620.0,150643.0,2019,145941645
2,Mar,150816.0,150832.0,150796.0,2019,146553073
3,Apr,151095.0,151020.0,151012.0,2019,147574087
4,May,151095.0,151084.0,151074.0,2019,148673869


In [ ]:
final_df.dtypes

,0
Month,object
Initial,float64
1st_revision,float64
2nd_revision,float64
Year,object
Final Value,object


In [ ]:
final_df['QCEW'] = final_df['Final Value']
final_df.drop(columns=['Final Value'], inplace=True)

final_df["Initial"] = pd.to_numeric(final_df["Initial"], errors="coerce")
final_df["QCEW"] = pd.to_numeric(final_df["QCEW"], errors="coerce")

for col in ["Initial", "1st_revision", "2nd_revision"]:
    final_df[col] = final_df[col] * 1000


final_df["%_Deviation_from_QCEW"] = ((final_df["Initial"] - final_df["QCEW"]) / final_df["QCEW"]) * 100

final_df = final_df[["Year","Month", "Initial", "1st_revision", "2nd_revision", "QCEW", "%_Deviation_from_QCEW"]]
final_df.head()

,Year,Month,Initial,1st_revision,2nd_revision,QCEW,%_Deviation_from_QCEW
0,2019,Jan,150574000.0,150586000.0,150587000.0,145328097.0,3.609696
1,2019,Feb,150606000.0,150620000.0,150643000.0,145941645.0,3.196041
2,2019,Mar,150816000.0,150832000.0,150796000.0,146553073.0,2.908794
3,2019,Apr,151095000.0,151020000.0,151012000.0,147574087.0,2.385861
4,2019,May,151095000.0,151084000.0,151074000.0,148673869.0,1.628485


In [ ]:
final_df.to_excel("NFP_revisions_vs_QCEW.xlsx",index=False)